In [1]:
#importing packages
import os 
import zipfile 
import tensorflow as tf 
from tensorflow.keras.optimizers import RMSprop 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model 
from tensorflow.keras.applications.inception_v3 import InceptionV3 
from tensorflow.keras.optimizers import RMSprop 
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

In [2]:
# re-size all the images to this
IMAGE_SIZE = [128, 128]

train_path = 'data/Train'
valid_path = 'data/Test'


In [3]:
#training data directory and parameteres
trainDataGen = ImageDataGenerator(
                    rotation_range = 5,
                    width_shift_range = 0.1,
                    height_shift_range = 0.1,
                    rescale = 1.0/255,
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    horizontal_flip = False,
                    fill_mode = 'nearest',
                )

testDataGen = ImageDataGenerator(rescale=1.0/255,)


train_gen = trainDataGen.flow_from_directory(
                    "data/Train",
                    target_size=(128, 128),
                    class_mode="categorical",
                    batch_size=64
                    )


Found 214603 images belonging to 108 classes.


In [4]:
#validation data directory and parameteres
val_gen = testDataGen.flow_from_directory(
                    "data/Test",
                    target_size=(128, 128),
                    class_mode="categorical",
                    batch_size=64
                    )


Found 32400 images belonging to 108 classes.


In [5]:
#building base model
base_model = InceptionV3(input_shape = IMAGE_SIZE + [3],  
                                include_top = False,  
                                weights = 'imagenet') 
for layer in base_model.layers: 
  layer.trainable = False
  
#stop training is model accuracy reached 99% 
class myCallback(tf.keras.callbacks.Callback): 
  def on_epoch_end(self, epoch, logs={}): 
    if(logs.get('acc')>0.99): 
      self.model.stop_training = True

In [6]:
# code 
x = layers.Flatten()(base_model.output) 

x = layers.BatchNormalization()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(256,activation='relu')(x) 
x = layers.Dropout(0.1)(x)  #dropout is decreased after observation of resutls on dataset
x = layers.BatchNormalization()(x)

x = layers.Dense(108, activation='softmax')(x)            
  
model = Model( base_model.input, x)  

In [7]:
#compiling the model
opt = tf.keras.optimizers.Adam()
model.compile(optimizer = opt,loss = 'categorical_crossentropy',metrics = ['acc']) # was using binary before

In [8]:
#if no change in validation loss - learning rate will be decreased
anne = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, verbose=1, min_lr=1e-3)
checkpoint = ModelCheckpoint('model.h5', verbose=1, save_best_only=True)

In [ ]:
#fitting the model  
incep = model.fit( 
            train_gen, 
            validation_data = val_gen, 
            steps_per_epoch = 3353, 
            epochs = 30, 
            validation_steps = 506) 

Epoch 1/30
3353/3353 [==============================] - 1502s 448ms/step - loss: 1.2727 - acc: 0.5959 - val_loss: 0.8233 - val_acc: 0.7137
Epoch 2/30
3353/3353 [==============================] - 1675s 499ms/step - loss: 0.9221 - acc: 0.6831 - val_loss: 0.6909 - val_acc: 0.7505
Epoch 3/30
3353/3353 [==============================] - 1709s 510ms/step - loss: 0.8324 - acc: 0.7089 - val_loss: 0.6584 - val_acc: 0.7559
Epoch 4/30
3353/3353 [==============================] - 1669s 498ms/step - loss: 0.7801 - acc: 0.7243 - val_loss: 0.6117 - val_acc: 0.7727
Epoch 5/30
3353/3353 [==============================] - 1651s 492ms/step - loss: 0.7420 - acc: 0.7363 - val_loss: 0.5865 - val_acc: 0.7794
Epoch 6/30
3353/3353 [==============================] - 1726s 515ms/step - loss: 0.7144 - acc: 0.7442 - val_loss: 0.5736 - val_acc: 0.7845
Epoch 7/30
3353/3353 [==============================] - 1662s 496ms/step - loss: 0.6894 - acc: 0.7501 - val_loss: 0.5575 - val_acc: 0.7912
Epoch 8/30
3353/3353 [=====

In [ ]:
#saving the model
model.save("IncepModel.h5")

In [ ]:
#plotting the graph
%matplotlib inline
accu=incep.history['acc']
val_acc=incep.history['val_acc']
loss=incep.history['loss']
val_loss=incep.history['val_loss']

epochs=range(len(accu)) #No. of epochs



In [ ]:
import matplotlib.pyplot as plt
plt.plot(epochs,accu,'r',label='Training Accuracy')
plt.plot(epochs,val_acc,'g',label='Testing Accuracy')

plt.savefig('training_accuracy_mix_incep.png')
plt.legend()
plt.figure()

#Plot training and validation loss per epoch
plt.plot(epochs,loss,'r',label='Training Loss')
plt.plot(epochs,val_loss,'g',label='Testing Loss')

plt.savefig('training_loss_mix_incep.png')
plt.legend()
plt.show()

